<a href="https://colab.research.google.com/github/imk1raa/imk1raa/blob/main/kira_afremov_ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SOURCE

In [78]:
%load_ext tensorboard
import datetime, os
import tensorflow as tf
import tensorflow_datasets as tfds


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


DOWNLOADING & PREPARING DATASET

In [79]:
(ds_train, ds_test), ds_info = tfds.load('mnist',
          split=['train', 'test'],
          shuffle_files=True,
          as_supervised=True,
          with_info=True,)

num_of_classes = ds_info.features['label'].num_classes
image_shape = ds_info.features['image'].shape
train_size = ds_info.splits['train'].num_examples
test_size = ds_info.splits['test'].num_examples


print(num_of_classes)
print(image_shape)
print(train_size)
print(test_size)


10
(28, 28, 1)
60000
10000


DATA PREPROCESING

In [80]:
def normalize_img(image, label):
    "Normalizes images: `uint8` -> `float32`"
    return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(32)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)


ds_test = ds_test.map(
     normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

DEEP ELU NETWORK

In [92]:
layers = [
        tf.keras.layers.Flatten(input_shape=image_shape),

        tf.keras.layers.Dense(87),
        tf.keras.layers.Activation('elu'),

        tf.keras.layers.Dense(30),
        tf.keras.layers.Activation('elu'),

        tf.keras.layers.Dense(64),
        tf.keras.layers.Activation('elu'),

        tf.keras.layers.Dense(37),
        tf.keras.layers.Activation('elu'),

        tf.keras.layers.Dense(num_of_classes),
        tf.keras.layers.Softmax()
        ]

DEFINE SEQUANTIAL MODEL

In [93]:
model = tf.keras.models.Sequential(layers)

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.summary()

Model: "sequential_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_26 (Flatten)                 │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_135 (Dense)                    │ (None, 87)                  │          68,295 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_101 (Activation)          │ (None, 87)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_136 (Dense)                    │ (None, 30)                  │           2,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_102 (Activation)          │ (None, 30)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_137 (Dense)                    │ (None, 64)                  │           1,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_103 (Activation)          │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_138 (Dense)                    │ (None, 37)                  │           2,405 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_104 (Activation)          │ (None, 37)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_139 (Dense)                    │ (None, 10)                  │             380 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ softmax_25 (Softmax)                 │ (None, 10)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 75,704 (295.72 KB)

 Trainable params: 75,704 (295.72 KB)

 Non-trainable params: 0 (0.00 B)

In [94]:
model.fit(ds_train,
          epochs=50,
          validation_data=ds_test)

Epoch 1/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 0.4266 - sparse_categorical_accuracy: 0.8718 - val_loss: 0.1596 - val_sparse_categorical_accuracy: 0.9509
Epoch 2/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.1468 - sparse_categorical_accuracy: 0.9564 - val_loss: 0.1159 - val_sparse_categorical_accuracy: 0.9641
Epoch 3/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.1015 - sparse_categorical_accuracy: 0.9694 - val_loss: 0.1118 - val_sparse_categorical_accuracy: 0.9668
Epoch 4/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.0817 - sparse_categorical_accuracy: 0.9752 - val_loss: 0.0993 - val_sparse_categorical_accuracy: 0.9695
Epoch 5/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.0691 - sparse_categorical_accuracy: 0.9788 - val_loss: 0.0902 - val_sparse_categorical_accuracy: 0.9739
Epoch 6/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.0602 - sparse_categorical_accuracy: 0.9808 - val_loss: 0.1059 - val_sparse_categorical_accuracy